In [2]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import os
import datetime
import pandas as pd
from IPython.display import display

In [3]:
from base64 import b64decode
from io import BytesIO

from IPython import get_ipython
from IPython.core.magic import register_cell_magic
import PIL


@register_cell_magic
def capture_png(line, cell):
    get_ipython().run_cell_magic(
        'capture',
        ' --no-stderr --no-stdout result',
        cell
    )
    out_paths = line.strip().split(' ')
    for output in result.outputs:
        data = output.data
        if 'image/png' in data:
            path = out_paths.pop(0)
            if not path:
                raise ValueError('Too few paths given!')
            png_bytes = data['image/png']
            if isinstance(png_bytes, str):
                png_bytes = b64decode(png_bytes)
            assert isinstance(png_bytes, bytes)
            bytes_io = BytesIO(png_bytes)
            image = PIL.Image.open(bytes_io)
            image.save(path, 'png')

In [4]:
#Read in Precip data files

root_dir = '/Users/meghan/Lemhi_Main/Lemhi/CONUS404/CONUS404_Data/C404_area_avgs/'
LRidge = nc.Dataset(os.path.join(root_dir, 'lemhiridge.nc'),'r')
BLower =  nc.Dataset(os.path.join(root_dir, 'beaverheadlower.nc'),'r')
BUpper= nc.Dataset(os.path.join(root_dir, 'beaverheadupper.nc'),'r')
LELower= nc.Dataset(os.path.join(root_dir, 'lemhieastlower.nc'),'r')
LEUpper= nc.Dataset(os.path.join(root_dir, 'lemhieastupper.nc'),'r')
PNorth= nc.Dataset(os.path.join(root_dir, 'pahsimeroinorth.nc'),'r')
JLMeadow= nc.Dataset(os.path.join(root_dir, 'jlmeadow.nc'),'r')
Schwartz= nc.Dataset(os.path.join(root_dir, 'schwartz.nc'),'r')
Gunsight= nc.Dataset(os.path.join(root_dir, 'gunsight.nc'),'r')
Moonshine= nc.Dataset(os.path.join(root_dir, 'moonshine.nc'),'r')


In [5]:
LRidge_Time = LRidge.variables['Time'][:]
LRidge_AS_LWC = LRidge.variables['AS_LWC'][:]
LRidge_Precip = LRidge.variables['PREC_ACC_NC'][:]
LRidge_Temp = LRidge.variables['AS_Tc'][:]

BLower_Time = BLower.variables['Time'][:]
BLower_AS_LWC = BLower.variables['AS_LWC'][:]
BLower_Precip = BLower.variables['PREC_ACC_NC'][:]
BLower_Temp = BLower.variables['AS_Tc'][:]

BUpper_Time = BUpper.variables['Time'][:]
BUpper_AS_LWC = BUpper.variables['AS_LWC'][:]
BUpper_GS_LWC = BUpper.variables['GS_LWC'][:]
BUpper_Precip = BUpper.variables['PREC_ACC_NC'][:]
BUpper_Temp = BUpper.variables['AS_Tc'][:]
BUpper_GS_Temp = BUpper.variables['GS_Tc'][:]

LEUpper_Time = LEUpper.variables['Time'][:]
LEUpper_AS_LWC = LEUpper.variables['AS_LWC'][:]
LEUpper_GS_LWC = LEUpper.variables['GS_LWC'][:]
LEUpper_Precip = LEUpper.variables['PREC_ACC_NC'][:]
LEUpper_Temp = LEUpper.variables['AS_Tc'][:]
LEUpper_GS_Temp = LEUpper.variables['GS_Tc'][:]

LELower_Time = LELower.variables['Time'][:]
LELower_AS_LWC = LELower.variables['AS_LWC'][:]
LELower_GS_LWC = LELower.variables['GS_LWC'][:]
LELower_Precip = LELower.variables['PREC_ACC_NC'][:]
LELower_Temp = LELower.variables['AS_Tc'][:]
LELower_GS_Temp = LELower.variables['GS_Tc'][:]

PNorth_Time = PNorth.variables['Time'][:]
PNorth_AS_LWC = PNorth.variables['AS_LWC'][:]
PNorth_GS_LWC = PNorth.variables['GS_LWC'][:]
PNorth_Precip = PNorth.variables['PREC_ACC_NC'][:]
PNorth_Temp = PNorth.variables['AS_Tc'][:]
PNorth_GS_Temp = PNorth.variables['GS_Tc'][:]

JLMeadow_Time = JLMeadow.variables['Time'][:]
JLMeadow_AS_LWC = JLMeadow.variables['AS_LWC'][:]
JLMeadow_GS_LWC = JLMeadow.variables['GS_LWC'][:]
JLMeadow_Precip = JLMeadow.variables['PREC_ACC_NC'][:]
JLMeadow_Temp = JLMeadow.variables['AS_Tc'][:]
JLMeadow_GS_Temp = JLMeadow.variables['GS_Tc'][:]

Schwartz_Time = Schwartz.variables['Time'][:]
Schwartz_AS_LWC = Schwartz.variables['AS_LWC'][:]
Schwartz_GS_LWC = Schwartz.variables['GS_LWC'][:]
Schwartz_Precip = Schwartz.variables['PREC_ACC_NC'][:]
Schwartz_Temp = Schwartz.variables['AS_Tc'][:]
Schwartz_GS_Temp = Schwartz.variables['GS_Tc'][:]

Gunsight_Time = Gunsight.variables['Time'][:]
Gunsight_AS_LWC = Gunsight.variables['AS_LWC'][:]
Gunsight_GS_LWC = Gunsight.variables['GS_LWC'][:]
Gunsight_Precip = Gunsight.variables['PREC_ACC_NC'][:]
Gunsight_GS_Temp = Gunsight.variables['GS_Tc'][:]

Moonshine_Time = Moonshine.variables['Time'][:]
Moonshine_AS_LWC = Moonshine.variables['AS_LWC'][:]
Moonshine_GS_LWC = Moonshine.variables['GS_LWC'][:]
Moonshine_Precip = Moonshine.variables['PREC_ACC_NC'][:]
Moonshine_Temp = Moonshine.variables['AS_Tc'][:]
Moonshine_GS_Temp = Moonshine.variables['GS_Tc'][:]

In [5]:
#Lemhi Ridge Precip with Beaverhead Lower Criteria
precip_df = pd.DataFrame({'time':LRidge_Time, 'precip' :LRidge_Precip})
criteria_df = pd.DataFrame({'time':BLower_Time, 'temp':BLower_Temp,
                             'lwc':BLower_AS_LWC})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
    _year_df = _odf[_odf.index.year == year]
    _precip_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    _crit_df = _precip_df[_precip_df['temp'] < -6]
    _crit1_df = _crit_df[_crit_df['temp'] > -18]
    _crit2_df = _crit1_df[_crit1_df['lwc'] > 0.01]
    return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        #'PGW LWC > 0.01 g/kg & -18 < Temp < -6 \xb0C':(len(_crit_df)/len(_month_df))*100, 
        'Seedable Precip':(_crit2_df.precip.sum()),
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Beaverhead Lower'
df['Precip']='Lemhi Ridge'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,358.355968,139.257268
std,66.227821,49.554494
min,230.397927,46.404281
25%,300.525237,104.379813
50%,357.170288,133.188075
75%,398.212994,169.924417
max,528.150069,279.454119


In [5]:
#Lemhi Ridge Precip with Beaverhead Lower Criteria
precip_df = pd.DataFrame({'time':LRidge_Time, 'precip' :LRidge_Precip})
criteria_df = pd.DataFrame({'time':BLower_Time, 'temp':BLower_Temp,
                             'lwc':BLower_AS_LWC, 'totalprecip':BLower_Precip})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
    _year_df = _odf[_odf.index.year == year]
    _criteria_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    _crit_df = _criteria_df[_criteria_df['totalprecip'] < 0]
   
    return {
        'year':year, 
        'Total Precip':(_criteria_df.precip.sum()),
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Beaverhead Lower'
df['Precip']='Lemhi Ridge'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip
count,42.000000
mean,358.355968
std,66.227821
min,230.397927
25%,300.525237
50%,357.170288
75%,398.212994
max,528.150069


In [6]:
#JLMeadow Precip with Beaverhead Upper Criteria
precip_df = pd.DataFrame({'time':JLMeadow_Time, 'precip' :JLMeadow_Precip})
criteria_df = pd.DataFrame({'time':BUpper_Time, 'temp':BUpper_Temp,
                             'lwc':BUpper_AS_LWC})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
    _year_df = _odf[_odf.index.year == year]
    _precip_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    _crit_df = _precip_df[_precip_df['temp'] < -6]
    _crit1_df = _crit_df[_crit_df['temp'] > -18]
    _crit2_df = _crit1_df[_crit1_df['lwc'] > 0.01]
    return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        #'PGW LWC > 0.01 g/kg & -18 < Temp < -6 \xb0C':(len(_crit_df)/len(_month_df))*100, 
        'Seedable Precip':(_crit2_df.precip.sum()),
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Beaverhead Upper'
df['Precip']='JL Meadow'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,329.900886,127.767755
std,69.107869,42.407837
min,182.655200,49.802224
25%,287.813522,107.221759
50%,326.782979,122.602846
75%,365.688702,144.016417
max,475.874546,238.763282


In [7]:
#Schwartz Precip with Lemhi East Lower Criteria
precip_df = pd.DataFrame({'time':Schwartz_Time, 'precip' :Schwartz_Precip})
criteria_df = pd.DataFrame({'time':LELower_Time, 'temp':LELower_Temp,
                             'lwc':LELower_AS_LWC})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
    _year_df = _odf[_odf.index.year == year]
    _precip_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    _crit_df = _precip_df[_precip_df['temp'] < -6]
    _crit1_df = _crit_df[_crit_df['temp'] > -18]
    _crit2_df = _crit1_df[_crit1_df['lwc'] > 0.01]
    return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        #'PGW LWC > 0.01 g/kg & -18 < Temp < -6 \xb0C':(len(_crit_df)/len(_month_df))*100, 
        'Seedable Precip':(_crit2_df.precip.sum()),
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Lemhi East Lower'
df['Precip']='Schwartz Lake'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,393.910874,98.135156
std,79.259989,37.617382
min,213.289497,27.863043
25%,346.525031,71.429293
50%,376.265030,92.642288
75%,448.401615,117.943057
max,583.950139,190.537904


In [8]:
#Gunsight Peak Precip with Lemhi East Upper Criteria
precip_df = pd.DataFrame({'time':Gunsight_Time, 'precip' :Gunsight_Precip})
criteria_df = pd.DataFrame({'time':LEUpper_Time, 'temp':LEUpper_Temp,
                             'lwc':LEUpper_AS_LWC})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
    _year_df = _odf[_odf.index.year == year]
    _precip_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    _crit_df = _precip_df[_precip_df['temp'] < -6]
    _crit1_df = _crit_df[_crit_df['temp'] > -18]
    _crit2_df = _crit1_df[_crit1_df['lwc'] > 0.01]
    return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        #'PGW LWC > 0.01 g/kg & -18 < Temp < -6 \xb0C':(len(_crit_df)/len(_month_df))*100, 
        'Seedable Precip':(_crit2_df.precip.sum()),
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Lemhi East Upper'
df['Precip']='Gunsight Peak'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,248.402324,93.303055
std,62.552527,32.804664
min,113.938819,37.405948
25%,212.383613,69.361621
50%,243.315971,86.626309
75%,275.729360,115.166824
max,466.630831,179.702083


In [9]:
#Moonshine Precip with Pahsimeroi North Criteria
precip_df = pd.DataFrame({'time':Moonshine_Time, 'precip' :Moonshine_Precip})
criteria_df = pd.DataFrame({'time':PNorth_Time, 'temp':PNorth_Temp,
                             'lwc':PNorth_AS_LWC})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
    _year_df = _odf[_odf.index.year == year]
    _precip_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    _crit_df = _precip_df[_precip_df['temp'] < -6]
    _crit1_df = _crit_df[_crit_df['temp'] > -18]
    _crit2_df = _crit1_df[_crit1_df['lwc'] > 0.01]
    return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        #'PGW LWC > 0.01 g/kg & -18 < Temp < -6 \xb0C':(len(_crit_df)/len(_month_df))*100, 
        'Seedable Precip':(_crit2_df.precip.sum()),
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Pahsimeroi North'
df['Precip']='Moonshine'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,370.654340,185.754834
std,86.224117,65.043394
min,187.994788,65.174543
25%,316.463895,134.049430
50%,368.377258,182.909036
75%,410.326606,225.370875
max,566.318840,357.254321


In [10]:
#Gunsight Peak Precip with Lemhi East Upper Criteria
precip_df = pd.DataFrame({'time':Gunsight_Time, 'precip' :Gunsight_Precip})
criteria_df = pd.DataFrame({'time':LEUpper_Time, 'temp':LEUpper_Temp,
                             'lwc':LEUpper_AS_LWC})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
    _year_df = _odf[_odf.index.year == year]
    _precip_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    _crit_df = _precip_df[_precip_df['temp'] < -6]
    _crit1_df = _crit_df[_crit_df['temp'] > -18]
    _crit2_df = _crit1_df[_crit1_df['lwc'] > 0.01]
    return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        #'PGW LWC > 0.01 g/kg & -18 < Temp < -6 \xb0C':(len(_crit_df)/len(_month_df))*100, 
        'Seedable Precip':(_crit2_df.precip.sum()),
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Lemhi East Upper'
df['Precip']='Gunsight'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,248.402324,93.303055
std,62.552527,32.804664
min,113.938819,37.405948
25%,212.383613,69.361621
50%,243.315971,86.626309
75%,275.729360,115.166824
max,466.630831,179.702083


In [11]:
#Schwartz Precip with Lemhi East Lower Criteria
precip_df = pd.DataFrame({'time':Schwartz_Time, 'precip' :Schwartz_Precip})
criteria_df = pd.DataFrame({'time':LELower_Time, 'temp':LELower_Temp,
                             'lwc':LELower_AS_LWC})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
    _year_df = _odf[_odf.index.year == year]
    _precip_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    _crit_df = _precip_df[_precip_df['temp'] < -6]
    _crit1_df = _crit_df[_crit_df['temp'] > -18]
    _crit2_df = _crit1_df[_crit1_df['lwc'] > 0.01]
    return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        #'PGW LWC > 0.01 g/kg & -18 < Temp < -6 \xb0C':(len(_crit_df)/len(_month_df))*100, 
        'Seedable Precip':(_crit2_df.precip.sum()),
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Lemhi East Lower'
df['Precip']='Schwartz'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,393.910874,98.135156
std,79.259989,37.617382
min,213.289497,27.863043
25%,346.525031,71.429293
50%,376.265030,92.642288
75%,448.401615,117.943057
max,583.950139,190.537904


In [6]:
#Beaverhead Lower Total Precip
precip_df = pd.DataFrame({'time':LRidge_Time, 'precip' :LRidge_Precip})
criteria_df = pd.DataFrame({'time':BLower_Time, 'temp':BLower_Temp,
                             'lwc':BLower_AS_LWC})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
    _year_df = _odf[_odf.index.year == year]
    _precip_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    _crit_df = _precip_df[_precip_df['temp'] < -6]
    _crit1_df = _crit_df[_crit_df['temp'] > -18]
    _crit2_df = _crit1_df[_crit1_df['lwc'] > 0.01]
    return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        #'PGW LWC > 0.01 g/kg & -18 < Temp < -6 \xb0C':(len(_crit_df)/len(_month_df))*100, 
        'Seedable Precip':(_crit2_df.precip.sum()),
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Beaverhead Lower'
df['Precip']='Lemhi Ridge'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip,Seedable Precip
count,42.000000,42.000000
mean,358.355968,139.257268
std,66.227821,49.554494
min,230.397927,46.404281
25%,300.525237,104.379813
50%,357.170288,133.188075
75%,398.212994,169.924417
max,528.150069,279.454119


In [ ]:
#Total Precip

In [17]:
#Beaverhead Lower Total Precip
precip_df = pd.DataFrame({'time':LRidge_Time, 'precip' :LRidge_Precip})
criteria_df = pd.DataFrame({'time':BLower_Time, 'temp':BLower_Temp,
                             'lwc':BLower_AS_LWC, 'totalprecip':BLower_Precip})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
    _fall_year_df = _odf[_odf.index.year == year]
    _fall_year_df = _fall_year_df[_fall_year_df.index.month>9]
    _spring_year_df = _odf[_odf.index.year == year+1]
    _spring_year_df = _spring_year_df[_spring_year_df.index.month<9]
    _wy_df=pd.concat([_fall_year_df, _spring_year_df])
    # _criteria_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    # _precip_df = _criteria_df[_criteria_df['totalprecip'] > 0]
    _precip_df = _wy_df[_wy_df['totalprecip'] > 0]
   
   
    return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        }

def collect_seasons():
    # df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_wy_df) for _wy_df in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Beaverhead Lower'
df['Precip']='Lemhi Ridge'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip
count,42.000000
mean,470.079248
std,104.558021
min,0.000000
25%,441.899324
50%,474.268060
75%,517.526842
max,652.196700


In [18]:
#Beaverhead Upper Total Precip
precip_df = pd.DataFrame({'time':JLMeadow_Time, 'precip' :JLMeadow_Precip})
criteria_df = pd.DataFrame({'time':BUpper_Time, 'temp':BUpper_Temp,
                             'lwc':BUpper_AS_LWC, 'totalprecip':BUpper_Precip})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
   _fall_year_df = _odf[_odf.index.year == year]
   _fall_year_df = _fall_year_df[_fall_year_df.index.month>9]
   _spring_year_df = _odf[_odf.index.year == year+1]
   _spring_year_df = _spring_year_df[_spring_year_df.index.month<9]
   _wy_df=pd.concat([_fall_year_df, _spring_year_df])
    # _criteria_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    # _precip_df = _criteria_df[_criteria_df['totalprecip'] > 0]
   _precip_df = _wy_df[_wy_df['totalprecip'] > 0]
   
   return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Beaverhead Lower'
df['Precip']='Lemhi Ridge'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip
count,42.000000
mean,467.517190
std,121.827913
min,0.000000
25%,399.486739
50%,463.037131
75%,570.069659
max,659.148177


In [19]:
#Lemhi East Lower Total Precip
precip_df = pd.DataFrame({'time':Schwartz_Time, 'precip' :Schwartz_Precip})
criteria_df = pd.DataFrame({'time':LELower_Time, 'temp':LELower_Temp,
                             'lwc':LELower_AS_LWC, 'totalprecip':LELower_Precip})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
   _fall_year_df = _odf[_odf.index.year == year]
   _fall_year_df = _fall_year_df[_fall_year_df.index.month>9]
   _spring_year_df = _odf[_odf.index.year == year+1]
   _spring_year_df = _spring_year_df[_spring_year_df.index.month<9]
   _wy_df=pd.concat([_fall_year_df, _spring_year_df])
    # _criteria_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    # _precip_df = _criteria_df[_criteria_df['totalprecip'] > 0]
   _precip_df = _wy_df[_wy_df['totalprecip'] > 0]
   
   return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Beaverhead Lower'
df['Precip']='Lemhi Ridge'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip
count,42.000000
mean,534.123933
std,128.554461
min,0.000000
25%,469.794243
50%,530.145432
75%,623.127102
max,768.698839


In [20]:
#Lemhi East Upper Total Precip
precip_df = pd.DataFrame({'time':Gunsight_Time, 'precip' :Gunsight_Precip})
criteria_df = pd.DataFrame({'time':LEUpper_Time, 'temp':LEUpper_Temp,
                             'lwc':LEUpper_AS_LWC, 'totalprecip':LEUpper_Precip})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(year=1980):
   _fall_year_df = _odf[_odf.index.year == year]
   _fall_year_df = _fall_year_df[_fall_year_df.index.month>9]
   _spring_year_df = _odf[_odf.index.year == year+1]
   _spring_year_df = _spring_year_df[_spring_year_df.index.month<9]
   _wy_df=pd.concat([_fall_year_df, _spring_year_df])
    # _criteria_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    # _precip_df = _criteria_df[_criteria_df['totalprecip'] > 0]
   _precip_df = _wy_df[_wy_df['totalprecip'] > 0]
   
   return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        }

def collect_seasons():
    df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons()

df['Region']='Beaverhead Lower'
df['Precip']='Lemhi Ridge'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip
count,42.000000
mean,343.921606
std,90.508602
min,0.000000
25%,308.475540
50%,339.686502
75%,388.569003
max,531.499840


In [22]:
#Pahsimeroi North Total Precip
precip_df = pd.DataFrame({'time':Moonshine_Time, 'precip' :Moonshine_Precip})
criteria_df = pd.DataFrame({'time':PNorth_Time, 'temp':PNorth_Temp,
                             'lwc':PNorth_AS_LWC, 'totalprecip':PNorth_Precip})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

def yearly_seasonal_average(_odf, year=1980):
   _fall_year_df = _odf[_odf.index.year == year]
   _fall_year_df = _fall_year_df[_fall_year_df.index.month>9]
   _spring_year_df = _odf[_odf.index.year == year+1]
   _spring_year_df = _spring_year_df[_spring_year_df.index.month<9]
   _wy_df=pd.concat([_fall_year_df, _spring_year_df])
    # _criteria_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
    # _precip_df = _criteria_df[_criteria_df['totalprecip'] > 0]
   _precip_df = _wy_df[_wy_df['totalprecip'] > 0]
   
   return {
        'year':year, 
        'Total Precip':(_precip_df.precip.sum()),
        }

def collect_seasons(_odf):
    df = pd.DataFrame.from_records([yearly_seasonal_average(_odf, year=_year) for _year in _odf.index.year.unique()])
    df.set_index('year', inplace=True, drop=True)
    return df

df = collect_seasons(_odf)

df['Region']='Beaverhead Lower'
df['Precip']='Lemhi Ridge'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip
count,42.000000
mean,480.017357
std,122.167589
min,0.000000
25%,412.003014
50%,461.246108
75%,575.830928
max,701.058518


In [23]:
#Lemhi East Upper Total Precip
precip_df = pd.DataFrame({'time':Gunsight_Time, 'precip' :Gunsight_Precip})
criteria_df = pd.DataFrame({'time':LEUpper_Time, 'temp':LEUpper_Temp,
                             'lwc':LEUpper_AS_LWC, 'totalprecip':LEUpper_Precip})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

# def yearly_seasonal_average(year=1980):
#     _year_df = _odf[_odf.index.year == year]
#     _criteria_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
#     _criteria_df = _criteria_df[_criteria_df['totalprecip'] > 0]
   
#     return {
#         'year':year, 
#         'Total Precip':(_criteria_df.precip.sum()),
#         }

# def collect_seasons():
#     df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
#     df.set_index('year', inplace=True, drop=True)
#     return df

# df = collect_seasons()
df = collect_seasons(_odf)
df['Region']='Beaverhead Lower'
df['Precip']='Lemhi Ridge'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip
count,42.000000
mean,343.921606
std,90.508602
min,0.000000
25%,308.475540
50%,339.686502
75%,388.569003
max,531.499840


In [24]:
#Lemhi East Lower Total Precip
precip_df = pd.DataFrame({'time':Schwartz_Time, 'precip' :Schwartz_Precip})
criteria_df = pd.DataFrame({'time':LELower_Time, 'temp':LELower_Temp,
                             'lwc':LELower_AS_LWC, 'totalprecip':LELower_Precip})

precip_df.set_index('time', drop=True, inplace=True) 
criteria_df.set_index('time', drop=True, inplace=True)

_odf = criteria_df.merge(precip_df, on = 'time') #joins two dataframes precip and critiera

_odf['time'] = pd.to_datetime(_odf.index, origin=datetime.datetime(1979,10,1), unit='m')#converts minutes to datetime
_odf.set_index('time', drop=True, inplace=True) #creates a true timeseries dataframe *magic that makes code below work*

# def yearly_seasonal_average(year=1980):
#     _year_df = _odf[_odf.index.year == year]
#     _criteria_df = pd.concat(_year_df[_year_df.index.month == mm] for mm in [11,12,1,2,3,4])
#     _criteria_df = _criteria_df[_criteria_df['totalprecip'] > 0]
   
#     return {
#         'year':year, 
#         'Total Precip':(_criteria_df.precip.sum()),
#         }

# def collect_seasons():
#     df = pd.DataFrame.from_records([yearly_seasonal_average(year=_year) for _year in _odf.index.year.unique()])
#     df.set_index('year', inplace=True, drop=True)
#     return df

df = collect_seasons(_odf)

df['Region']='Beaverhead Lower'
df['Precip']='Lemhi Ridge'
df['Year']=df.index
df.set_index(['Region', 'Precip','Year'], inplace=True, drop=True)
#display(df)
df.describe()




,Total Precip
count,42.000000
mean,534.123933
std,128.554461
min,0.000000
25%,469.794243
50%,530.145432
75%,623.127102
max,768.698839
